<a href="https://colab.research.google.com/github/parth06/Fake-News-Challenge/blob/master/Neural%20Net/LSTM_%26_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/parth06/Fake-News-Challenge.git

fatal: destination path 'Fake-News-Challenge' already exists and is not an empty directory.


In [3]:
cd Fake-News-Challenge/

/content/Fake-News-Challenge


In [0]:
!git submodule init
!git submodule update

In [0]:
import nltk

In [6]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
!mkdir data/glove
!mkdir data/glove_twitter

In [35]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!wget http://nlp.stanford.edu/data/glove.6B.zip

!unzip glove.twitter.27B.zip -d data/glove_twitter/
!unzip glove.6B.zip -d data/glove

--2019-07-26 12:06:55--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2019-07-26 12:06:55--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2019-07-26 12:06:55--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [0]:
!rm glove.twitter.27B.zip
!rm glove.6B.zip

In [7]:
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from keras import callbacks
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import punctuation

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints

tf_gpu_options = tf.GPUOptions(allow_growth = True) # per_process_gpu_memory_fraction=0.12,
tf_session = tf.Session(config=tf.ConfigProto(gpu_options=tf_gpu_options))
tf.keras.backend.set_session(tf_session)

#biodirectional embedding


Using TensorFlow backend.


In [0]:
# Specify the folder locations
#W2V_DIR = './data/GoogleNews-vectors-negative300.bin' #
GloVe_DIR = './data/glove_twitter/glove.twitter.27B.50d.txt'
#the data directory
DATA_DIR = './data/old'
# These are some hyperparameters that can be tuned
MAX_SENT_LEN = 300 #75(0.68), 150, 300 700(90% but too time comsuming)
MAX_VOCAB_SIZE = 28000 #vocabulary
LSTM_DIM = 50#len(embd[0])
EMBEDDING_DIM = 50 #50 for GloVe 300 for w2v
BATCH_SIZE = 128
N_EPOCHS = 40 #40

In [0]:
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

In [0]:
# Read the text files of fnc data
bodies = pd.read_csv(DATA_DIR + '/body_id.csv')
    
train_df = pd.read_csv(DATA_DIR + '/train.csv')

validation_df = pd.read_csv(DATA_DIR + '/validation.csv')

test_df = pd.read_csv(DATA_DIR + '/test.csv')


In [0]:
train_df.replace('unrelated',1,True)
train_df.replace('agree',2,True)
train_df.replace('disagree',3,True)
train_df.replace('discuss',4,True)

validation_df.replace('unrelated',1,True)
validation_df.replace('agree',2,True)
validation_df.replace('disagree',3,True)
validation_df.replace('discuss',4,True)

In [0]:
#Unrelated:1 Related:0
def f(row):
    if row['Stance']==4:
      return 1
    else:
      return 0

train_df['Classify'] = train_df.apply(f, axis=1)
validation_df['Classify'] = validation_df.apply(f, axis=1)

In [0]:
combine_df_train = train_df.join(bodies.set_index('Body ID'), on='Body ID')
combine_df_validation = validation_df.join(bodies.set_index('Body ID'), on='Body ID')
combine_df_test = test_df.join(bodies.set_index('Body ID'), on='Body ID')

In [0]:
# Pre-processing involves removal of puctuations and converting text to lower case
word_seq_head_train = [text_to_word_sequence(head) for head in combine_df_train['Headline']]
word_seq_bodies_train = [text_to_word_sequence(body) for body in combine_df_train['articleBody']]
word_seq_head_validation = [text_to_word_sequence(head) for head in combine_df_validation['Headline']]
word_seq_bodies_validation = [text_to_word_sequence(body) for body in combine_df_validation['articleBody']]
word_seq_head_test = [text_to_word_sequence(head) for head in combine_df_test['Headline']]
word_seq_bodies_test = [text_to_word_sequence(body) for body in combine_df_test['articleBody']]

In [15]:
print('90th Percentile Sentence of headline:', np.percentile([len(seq) for seq in word_seq_head_train], 90))
print('90th Percentile Sentence of body:', np.percentile([len(seq) for seq in word_seq_bodies_train], 50))

90th Percentile Sentence of headline: 16.0
90th Percentile Sentence of body: 308.0


In [0]:
word_seq = []
for i in range(len(word_seq_head_train)):
    word_seq.append(word_seq_head_train[i])
for i in range(len(word_seq_bodies_train)):
    word_seq.append(word_seq_bodies_train[i])

# for i in range(len(word_seq_head_validation)):
#     word_seq.append(word_seq_head_validation[i])
# for i in range(len(word_seq_bodies_validation)):
#     word_seq.append(word_seq_bodies_validation[i])
    
# for i in range(len(word_seq_head_test)):
#     word_seq.append(word_seq_head_test[i])
# for i in range(len(word_seq_bodies_test)):
#     word_seq.append(word_seq_bodies_test[i])

In [17]:
filter_list = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
MAX_VOCAB_SIZE = 5000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters=filter_list)
tokenizer.fit_on_texts([seq for seq in word_seq])
#because it only includes unique words(tokens)

print("Number of words in vocabulary:", len(tokenizer.word_index))

Number of words in vocabulary: 25128


In [0]:
# embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0

# for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
#   try:
#       embeddings_vector = embeddings[word]
#   except KeyError:
#       embeddings_vector = None
#   if embeddings_vector is not None:
#       embeddings_matrix[i] = embeddings_vector

# del embeddings

In [19]:
Xtrain_headline = tokenizer.texts_to_matrix(word_seq_head_train, mode='freq')
Xtrain_headline.shape

Xtrain_body = tokenizer.texts_to_matrix(word_seq_bodies_train, mode='freq')
Xtrain_body.shape

(40350, 5000)

In [20]:
Xval_headline = tokenizer.texts_to_matrix(word_seq_head_validation, mode='freq')
Xval_headline.shape

Xval_body = tokenizer.texts_to_matrix(word_seq_bodies_validation, mode='freq')
Xval_body.shape

(9622, 5000)

In [21]:
Xtest_headline = tokenizer.texts_to_matrix(word_seq_head_test, mode='freq')
Xtest_headline.shape

Xtest_body = tokenizer.texts_to_matrix(word_seq_bodies_test, mode='freq')
Xtest_body.shape

(25413, 5000)

In [22]:
model1_in = Input(shape=(5000,))
model1_out = (model1_in)
model1 = Model(model1_in, model1_out)

model2_in = Input(shape=(5000,))
model2_out = (model2_in)
model2 = Model(model2_in, model2_out)


concatenated = concatenate([model1_out, model2_out])
layer_1 = Dense(600, activation='relu', name='layer_1') (concatenated)
dropout_1 = Dropout(rate=0.5, name='dropout_1')(layer_1)
layer_2 =Dense(600,  activation='relu', name='layer_2') (dropout_1)
dropout_2 = Dropout(rate=0.5, name='dropout_2')(layer_2)
layer_3 =Dense(600,  activation='relu', name='layer_3') (dropout_2)
dropout_3 = Dropout(rate=0.5, name='dropout_3')(layer_3)
out1 =  Dense(2, activation='softmax', name='output_layer1')(dropout_3)
out2 = Dense(4, activation='softmax', name='output_layer2')(dropout_3)

losses = {
	"output_layer1": "categorical_crossentropy",
	"output_layer2": "categorical_crossentropy",
}
lossWeights = {"output_layer1": 0.25, "output_layer2": 0.75}
               
merged_model = Model([model1_in, model2_in], [out1,out2])
merged_model.compile(loss=losses, optimizer='adam', loss_weights=lossWeights,
metrics=['accuracy'])

# checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc',
# save_best_only=True, verbose=2)
# early_stopping = EarlyStopping(monitor="val_loss", patience=5)

W0726 12:00:36.275502 140477277935488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 12:00:36.278296 140477277935488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 12:00:36.296282 140477277935488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 12:00:36.319746 140477277935488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 12:00:36.334908 

In [23]:
merged_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5000)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 5000)         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 10000)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
layer_1 (Dense)                 (None, 600)          6000600     concatenate_1[0][0]              
__________

In [0]:
from keras.utils import plot_model
plot_model(merged_model, show_shapes=True)

In [0]:
y_train = combine_df_train['Stance']
y_val = combine_df_validation['Stance']
#Encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(y_train)
encoded_train = encoder_train.transform(y_train)
encoded_val = encoder_train.transform(y_val)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_train)
dummy_y_val = np_utils.to_categorical(encoded_val)

y1_train = train_df['Classify']
y1_val = validation_df['Classify']
#Encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(y1_train)
encoded_train = encoder_train.transform(y1_train)
encoded_val = encoder_train.transform(y1_val)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y1_train = np_utils.to_categorical(encoded_train)
dummy_y1_val = np_utils.to_categorical(encoded_val)

In [26]:
merged_model.fit([Xtrain_headline, Xtrain_body], y=[dummy_y1_train,dummy_y_train], batch_size=128, epochs=20, 
                 validation_data=([Xval_headline,Xval_body],[dummy_y1_val,dummy_y_val]),verbose=1, shuffle=True)

W0726 12:00:38.217474 140477277935488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40350 samples, validate on 9622 samples
Epoch 1/20
40350/40350 [==============================] - 12s 307us/step - loss: 0.5610 - output_layer1_loss: 0.3576 - output_layer2_loss: 0.6288 - output_layer1_acc: 0.8564 - output_layer2_acc: 0.7804 - val_loss: 0.4137 - val_output_layer1_loss: 0.2583 - val_output_layer2_loss: 0.4655 - val_output_layer1_acc: 0.8921 - val_output_layer2_acc: 0.8312
Epoch 2/20
40350/40350 [==============================] - 9s 230us/step - loss: 0.2169 - output_layer1_loss: 0.1343 - output_layer2_loss: 0.2444 - output_layer1_acc: 0.9467 - output_layer2_acc: 0.9139 - val_loss: 0.2977 - val_output_layer1_loss: 0.1792 - val_output_layer2_loss: 0.3372 - val_output_layer1_acc: 0.9226 - val_output_layer2_acc: 0.8716
Epoch 3/20
40350/40350 [==============================] - 9s 232us/step - loss: 0.1278 - output_layer1_loss: 0.0798 - output_layer2_loss: 0.1438 - output_layer1_acc: 0.9685 - output_layer2_acc: 0.9465 - val_loss: 0.2446 - val_output_layer1_loss: 0.16

In [0]:
y,y_test = merged_model.predict([Xtest_headline,Xtest_body])
y_test = np.argmax(y_test,axis=1)

In [28]:
ans = pd.DataFrame(y_test)
ans.replace(0,'unrelated',True)
ans.replace(1,'agree',True)
ans.replace(2,'disagree',True)
ans.replace(3,'discuss',True)
ans.head()

,0
0,discuss
1,unrelated
2,unrelated
3,unrelated
4,unrelated


**Simple LSTM**

In [0]:
# Combine headline and body together
word_seq_train = [list(i) for i in word_seq_head_train]
for i in range(len(word_seq_head_train)):
    word_seq_train[i].extend(word_seq_bodies_train[i]) 

word_seq_validation = [list(i) for i in word_seq_head_validation]
for i in range(len(word_seq_head_validation)):
    word_seq_validation[i].extend(word_seq_bodies_validation[i]) 
    
word_seq_test = [list(i) for i in word_seq_head_test]
for i in range(len(word_seq_head_test)):
    word_seq_test[i].extend(word_seq_bodies_test[i])

In [0]:
# Shorten the sentence to a fixed length
# Convert the sequence of words to sequnce of indices
X_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_train])
X_train = pad_sequences(X_train, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
y_train = combine_df_train['Stance']

X_val = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_validation])
X_val = pad_sequences(X_val, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
y_val = combine_df_validation['Stance']

In [0]:
# Convert the sequence of words to sequnce of indices
X_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_test])
X_test = pad_sequences(X_test, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

In [37]:
# Load the word2vec embeddings 
#embeddings = gensim.models.KeyedVectors.load_word2vec_format(GloVe_DIR, binary=True)

#GloVes Load
glove_input_file = GloVe_DIR
word2vec_output_file = 'glove.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
embeddings = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# Create an embedding matrix containing only the word's in our vocabulary
# If the word does not have a pre-trained embedding, then randomly initialize the embedding
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0
for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None
        #none: if words in sentence don't have pre-trained corresponding embedding, then error occurs

    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
        #if pre-trained word embedding exists，then let embeddings_matrix[i] is this embedding
        #Wi:the ith row of embeddings_matrix
        
del embeddings
#delete

In [39]:
# Build a sequential model by stacking neural net units 
model_1 = Sequential()
model_1.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embeddings_matrix], trainable=False, name='word_embedding_layer', #False
                          mask_zero=True))
#mask_zero is to deal with padding problem
#model.add(Bidirectional(LSTM(LSTM_DIM, return_sequences=False, name='lstm_layer1'))) #bi(lstm)
model_1.add(Bidirectional(LSTM(LSTM_DIM, return_sequences=False, name='Bidrectional_lstm_layer1')))
# model_1.add(Dropout(rate=0.8, name='dropout_1')) # Can try varying dropout rates, in paper suggest 0.8
# model_1.add(Dense(4, activation='softmax', name='output_layer'))
model_1.add(Dropout(rate=0.8, name='dropout_1'))
model_1.add(Activation(activation='relu', name='activation_1'))

#model.add(Dense(8, name='dense_2'))
# model.add(BatchNormalization(name='bn_2'))
model_1.add(Dropout(rate=0.5, name='dropout_2'))
model_1.add(Activation(activation='relu', name='activation_2'))

model_1.add(Dense(4, activation='softmax', name='output_layer'))
model_1.summary()

W0726 12:13:03.954757 140477277935488 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, None, 50)          1256450   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               40400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
_________________________________________________________________
output_layer (Dense)         (None, 4)                 404       
Total para

In [0]:
plot_model(model_1, to_file='bidirectional_model.png', show_layer_names=True, show_shapes=True)

In [0]:
!mkdir result
!mkdir result/lstm

In [0]:
filepath="./result/lstmBidirectional_lstm_150token_lr0.001_trainable_{epoch:02d}_{val_acc:.4f}.h5"
checkpoint = callbacks.ModelCheckpoint(filepath, 
                                       monitor='val_acc', 
                                       verbose=0, 
                                       save_best_only=True)
es = callbacks.EarlyStopping(monitor='val_acc', mode='max',patience=2)
callbacks_list1 = [checkpoint,es]

In [0]:
#before trainning model, we need to compile it
#use adam optimizer
model_1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
history_3 = model_1.fit(X_train, dummy_y_train,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(X_val, dummy_y_val),callbacks = callbacks_list1)

Train on 40350 samples, validate on 9622 samples
Epoch 1/40
40350/40350 [==============================] - 514s 13ms/step - loss: 0.9125 - acc: 0.7185 - val_loss: 0.8129 - val_acc: 0.7169
Epoch 2/40
40350/40350 [==============================] - 521s 13ms/step - loss: 0.8337 - acc: 0.7349 - val_loss: 0.7815 - val_acc: 0.7169
Epoch 3/40
40350/40350 [==============================] - 519s 13ms/step - loss: 0.7898 - acc: 0.7354 - val_loss: 0.7605 - val_acc: 0.7169


In [0]:
y = model_1.predict_classes(X_test)

In [0]:
ans = pd.DataFrame(y)

In [0]:
ans.head()

In [0]:
ans.replace(0,'unrelated',True)
ans.replace(1,'agree',True)
ans.replace(2,'disagree',True)
ans.replace(3,'discuss',True)

In [0]:
ans.head()

In [0]:
ans.to_csv("stance.csv",header=False,index=False)

In [0]:
test_df.head()

In [0]:
plt.plot(history_3.history['acc'])
plt.plot(history_3.history['val_acc'])
plt.title('model accuracy(Truncation: 150 Epoch: 10)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
import sys
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission

from utils.system import parse_params, check_version
def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    return y
  

competition_dataset = DataSet("competition_test")
y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

# Load/Precompute all features now


Reading dataset
Total stances: 25413
Total bodies: 904


In [0]:
#Run on competition dataset
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,ans[0])


Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    395    |    173    |    612    |    723    |
-------------------------------------------------------------
| disagree  |    125    |    60     |    223    |    289    |
-------------------------------------------------------------
|  discuss  |    216    |    209    |   2582    |   1457    |
-------------------------------------------------------------
| unrelated |    685    |    810    |   3560    |   13294   |
-------------------------------------------------------------
Score: 6750.0 out of 11651.25	(57.93369810106212%)


57.93369810106212

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
model_2 = Sequential()
model_2.add(Embedding(input_dim=len(tokenizer.word_index)+1, input_length = MAX_SENT_LEN,
                          output_dim=EMBEDDING_DIM,
                          weights = [embeddings_matrix], trainable= False, name='word_embedding_layer', 
                          mask_zero=True)) # trainable=True results in overfitting

model_2.add(LSTM(LSTM_DIM, return_sequences=True, name='lstm_layer')) # Can try Bidirectional-LSTM
model_2.add(Dropout(rate=0.8, name='dropout_1')) # Can try varying dropout rates, in paper suggest 0.2
model_2.add(Activation(activation='relu', name='activation_1'))
#output (batch_size, timesteps, input_dim)
model_2.add(Attention(step_dim=300))
model_2.add(Dense(4, activation='softmax', name='output_layer'))

W0715 18:41:18.151100 140300798273408 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [0]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 300, 50)           1393700   
_________________________________________________________________
lstm_layer (LSTM)            (None, 300, 100)          60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 100)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 300, 100)          0         
_________________________________________________________________
attention_2 (Attention)      (None, 100)               400       
_________________________________________________________________
output_layer (Dense)         (None, 4)                 404       
Total params: 1,454,904
Trainable params: 61,204
Non-trainable params: 1,393,700
_____________________________________________________________

In [0]:
optimizer = optimizers.Adam(lr=0.001) # Try a different learning rate

model_2.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [0]:
es = callbacks.EarlyStopping(monitor='val_acc', mode='max',patience=3)
callbacks_list2 = [es]

In [0]:
history2 = model_2.fit(X_train, dummy_y_train,
          batch_size=BATCH_SIZE,
          epochs=1,
          validation_data=(X_val, dummy_y_val),
          callbacks = callbacks_list2)

In [0]:
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('model accuracy(Truncation: 150 Epoch: 10)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
y = model_2.predict_classes(X_test)
ans = pd.DataFrame(y)
ans.head()

ans.replace(1,'unrelated',True)
ans.replace(2,'agree',True)
ans.replace(3,'disagree',True)
ans.replace(4,'discuss',True)

In [0]:
print("Scores on the test set")
report_score(actual,ans[0])


In [0]:
#https://www.kaggle.com/takuok/bidirectional-lstm-and-attention-lb-0-043